# Metadata in ATLAS Open Data

All datasets have associated metadata -- properties shared by all the events in the dataset. In the ATLAS Open Data, metadata includes things like a unique numerical dataset identifier, the production cross section, and the description of the physics processes included in the sample. These metadata must be used to properly normalize the MC simulation datasets when comparing to detector data, and they can also be used to search for relevant or related samples.

The metadata are available in a package called `atlasopenmagic` that is [available on pypi](http://pypi.org/project/atlasopenmagic/). Let's set it up and use it to explore the metadata of the most recent ATLAS Open Data release!

Note that the metadata we will be looking at are also [available on the web](https://opendata.atlas.cern/docs/data/for_education/13TeV25_metadata) in a big table, but using a python module will make it much easier to grab numbers that you need for an analysis in an automatic way.

In [ ]:
# First we install atlasopenmagic into our SWAN environment
# Notice that we need --user to avoid trying to install the package in a
# read-only file system This is a problem unique to SWAN; on binder or colab you
# won't need --user, but it doesn't hurt
%pip install --user atlasopenmagic

In [ ]:
# Now we have to do a little bit of work to make sure that atlasopenmagic is
# available in our python path This is because SWAN by default does not include
# the local package installation area in the PYTHONPATH Again, this is not
# necessary on binder or colab - there you can remove these lines if you like,
# though they don't do any harm
import sys
import os
sys.path += [ f'{os.environ["HOME"]}/.local/lib/python{sys.version_info.major}.{sys.version_info.minor}/site-packages' ]

# Now we can safely import atlasopenmagic
import atlasopenmagic as atom

In [ ]:
# Now let's see what releases are available to us
atom.available_releases()

In [ ]:
# And let's use the latest release of Open Data for Outreach and Education
atom.set_release('2025e-13tev-beta')

If you want to immediately find out what samples are available, you can use:
```python
atom.available_datasets()
```

That will return a long list of integer "dataset identifiers". There aren't good rules about what dataset gets assigned what integer, so they're most useful for software that needs to keep lists of samples. In this notebook, we're going to use a sample that we already know exists; later on we'll show you how to identify the right samples for your use-case.

In [ ]:
# Now we can look at the metadata for a specific sample
atom.get_metadata(345060)
# Notice that the function here will accept either the dataset identifier or the
# "physics short", a short unique descriptor for the sample %% [markdown] That's
# a lot of metadata! Let's go through the fields a bit:

That's a lot of metadata! Let's go through the fields a bit:

* `dataset_number`: Unique identifier assigned to each dataset.
* `physics_short`: Short name with information regarding the content of the dataset.
* `cross_section_pb`: A [cross section](https://atlas.cern/glossary/cross-section) represents the probability of a particular interaction occurring, measured in picobarns (pb). It is a fundamental parameter that helps understanding the likelihood of specific particle interactions under given conditions. The cross section is usually what is returned by the generator - you need to multiply by the filter efficiency and k-factor (see below) to get a complete sample weight.
* `genFiltEff`: Measure of the effectiveness of the selection criteria applied to the data. It indicates the fraction of events that pass the filters applied during the data processing stages.
* `kFactor`: Multiplicative correction factor used to account for higher-order effects in theoretical calculations. It adjusts the leading-order theoretical predictions to better match the observed data by incorporating next-to-leading order (NLO) or next-to-next-to-leading order (NNLO) corrections.
* `nEvents`: Total count of the events in the (unskimmed) dataset.
* `sumOfWeights` (`sumOfWeightsSquared`): Sum of the event weights (or event weights squared) in the released dataset. Use this for normalization of the samples (for understanding the statistical power of the dataset).
* `generator`: Specifies the simulation software used to generate the data. Generators are described in more detail [here](https://opendata.atlas.cern/docs/documentation/monte_carlo/simulation_tools).
* `keywords`: Terms or phrases associated with the dataset that help to find specific datasets.
* `process`: Brief description of the physics process being studied. For instance, "H->γγ" denotes the Higgs boson decaying into two photons. Often this describes the [Feynman Diagram](https://cds.cern.ch/record/2759490/) of the highest energy interaction in the event.
* `job_path`: Link to the specific code or configuration files used to generate the sample. Sometimes these will be fairly easy to understand; in some cases they are quite complex and difficult for non-experts to understand.
* `description`: Longer description of the process that was generated to create the sample.
* `e_tag`: ATLAS software configuration (including software version) that was used to create the sample.
* `file_list`: The list of paths for the unskimmed files in the dataset.
* `skims`: The list of metadata for the skimmed files in the dataset. Only one skimmed or unskimmed version should be used at a time. Within the skims we have:
   * A file list for the skim.
   * The name of the skim. The exact selections are [described here](https://opendata.atlas.cern/docs/data/for_education/13TeV25_details)

In [ ]:
# Get an individual field of metadata
xsec = atom.get_metadata('345060', 'cross_section_pb')
print(xsec)

In [ ]:
# Keywords are a great way to find datasets that you're interested in. Let's see
# what keywords are available.
atom.available_keywords()

In [ ]:
# Now let's find datasets that match one of those keywords
atom.match_metadata(field='keywords',value='Higgs')

In [ ]:
# Now lets look for samples that have `W` somewhere in the process name
# Notice that this also catches things like `Wprime`!
atom.match_metadata(field='process',value='W')

In [ ]:
# We can also find samples that have a cross section near one we're interested in.
# By default, the matching is pretty tight (1%):
atom.match_metadata('cross_section_pb',0.001762)

In [ ]:
# But we can loosen the tolerance to 50% to see what other samples have similar
# cross sections.
atom.match_metadata('cross_section_pb',0.001762,float_tolerance=0.50)

The [atlasopenmagic documentation](https://pypi.org/project/atlasopenmagic/) has many more examples of searches; please feel free to play with the package yoourself, and [open an issue](https://github.com/atlas-outreach-data-tools/atlasopenmagic/issues) or a [pull request](https://github.com/atlas-outreach-data-tools/atlasopenmagic/pulls) if there is a feature that you would like to have available!

In case you would like to explore the metadata some more on your own, here are some things you might try:

* Go back to the list of samples that we found that match the `Higgs` keyword. Can you understand how each is different based on the names and metadata?
* Let's pretend we're going to do an analysis using the QCD jet spectrum. Try using the `QCD` keyword to identify the samples that you would want to use.
    * You probably found quite a few samples! Can you tell what the differences are between them based on the metadata?
* Let's pretend we're going to do a physics analysis in a dilepton final state. How can you identify all the samples that you are going to want? Can you think through all the physics processes that might be relevant and identify the corresponding samples?
* Try loading up the 13 TeV event generation open data metadata (the release is `2025r-evgen-13tev`). There are a LOT more samples available, so this will take a moment to load (typically around 20-30s).
    * What keywords are available? Are they different from the ones you found before?
    * Now try looking for the QCD dijet samples again. There are even more! What are the differences between them?
* Pick your favorite [ATLAS paper](https://twiki.cern.ch/twiki/bin/view/AtlasPublic/Publications) and see if you can identify the actual samples that were used in that paper in the Open Data. They aren't all available, but most of them are!